In [1]:
import pandas as pd
import numpy as np
import re
from datetime import datetime
from scipy import stats
from feature_engine.imputation import MeanMedianImputer

In [2]:
df = pd.read_csv("C:\\Users\\Ivand\\Repos\\banks\\data\\Bonds.csv")

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (31,57,58,59,60) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
df.columns

Index(['Unnamed: 0', 'Issuerx', 'Issue', 'Number', 'Currency', 'Date_trading',
       'Date_final', 'Exch_name', 'Bid_price', 'Ask_price', 'BA_spread',
       'Av_price', 'Opening_price', 'Min_price', 'Max_price', 'Last_price',
       'Av_weighted_price', 'Closing_price', 'Indicative_price',
       'Indicative_price_type', 'Turnover', 'Number_of_trades', 'Volume',
       'Coupon', 'YTM_ind', 'YTM_bid', 'YTM_ask', 'YTM_last', 'YTM_close',
       'Coupon_accum', 'Duration', 'Option_date', 'YTM_off_ind', 'YTM_off_bid',
       'YTM_off_ask', 'YTM_off_last', 'YTM_off_close', 'YTM_curr_ind',
       'YTM_curr_bid', 'YTM_curr_ask', 'YTM_curr_last', 'YTM_curr_close',
       'YTM_ind_main', 'Indicative_yield_type', 'Duration_option',
       'Trade_regime', 'ISIN', 'G_spread', 'T_spread', 'Date_first_trading',
       'State_bank_name', 'State_bank', 'Issuer', 'Days_to_maturity',
       'Total_days_to_maturity', 'Days_to_call', 'Callable', 'cnamer',
       'cregnum', 'cdreg', 'strcuraddr', 'G_spre

In [4]:
len(df.cnamer.unique())

77

In [5]:
df = df[(df.Issuerx != "вэб")&(df.Issuerx != "мбэс")]

In [46]:
df_short = df[["Date_trading", "Date_final", 'Exch_name', 'BA_spread',
   'Indicative_price','Indicative_price_type', 'Turnover', 'Number_of_trades', 'Volume',
       'Coupon', 'YTM_ind','Coupon_accum', 'Duration', 'Option_date','Trade_regime', 'ISIN',
       'Date_first_trading','Total_days_to_maturity','Days_to_maturity', 'Days_to_call', 'Callable','RF_interpolated',
       'G_spread_interpolated', "cregnum"]]
df_short = df_short[df_short.Exch_name != "Cbonds Estimation"]
df_short = df_short[df_short.Exch_name != "Cbonds Valuation"]

df_short

,Date_trading,Date_final,Exch_name,BA_spread,Indicative_price,Indicative_price_type,Turnover,Number_of_trades,Volume,Coupon,...,Trade_regime,ISIN,Date_first_trading,Total_days_to_maturity,Days_to_maturity,Days_to_call,Callable,RF_interpolated,G_spread_interpolated,cregnum
0,2016-07-26 00:00:00,2016-07-27 00:00:00,Московская Биржа,NaN,99.87,Market,NaN,NaN,NaN,0.115,...,Осн. режим: Облигации - Первый уровень,RU000A0JRNJ3,2015-01-05 00:00:00,569,1,NaN,0,0.100886,0.643114,2 306
1,2016-07-25 00:00:00,2016-07-27 00:00:00,Московская Биржа,NaN,99.87,Market,NaN,NaN,NaN,0.115,...,Осн. режим: Облигации - Первый уровень,RU000A0JRNJ3,2015-01-05 00:00:00,569,2,NaN,0,0.101171,0.294529,2 306
2,2016-07-22 00:00:00,2016-07-27 00:00:00,Московская Биржа,NaN,99.87,Market,NaN,NaN,NaN,0.115,...,Осн. режим: Облигации - Первый уровень,RU000A0JRNJ3,2015-01-05 00:00:00,569,5,NaN,0,0.101322,0.118178,2 306
3,2016-07-21 00:00:00,2016-07-27 00:00:00,Московская Биржа,NaN,99.87,Market,NaN,NaN,NaN,0.115,...,Осн. режим: Облигации - Первый уровень,RU000A0JRNJ3,2015-01-05 00:00:00,569,6,NaN,0,0.101107,0.100693,2 306
4,2016-07-20 00:00:00,2016-07-27 00:00:00,Московская Биржа,NaN,99.87,Market,NaN,NaN,NaN,0.115,...,Осн. режим: Облигации - Первый уровень,RU000A0JRNJ3,2015-01-05 00:00:00,569,7,NaN,0,0.101091,0.087609,2 306
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
552599,2015-01-12 00:00:00,2019-11-20 00:00:00,Московская Биржа,NaN,100.00,Market,NaN,NaN,NaN,0.091,...,Осн. режим: Облигации - Третий уровень,RU000A0JV052,2015-01-05 00:00:00,1780,1773,329.0,1,0.163116,-0.039916,1
552600,2015-01-09 00:00:00,2019-11-20 00:00:00,Московская Биржа,NaN,100.00,Market,NaN,NaN,NaN,0.091,...,Осн. режим: Облигации - Третий уровень,RU000A0JV052,2015-01-05 00:00:00,1780,1776,329.0,1,0.155303,-0.032103,1
552601,2015-01-08 00:00:00,2019-11-20 00:00:00,Московская Биржа,NaN,100.00,Market,NaN,NaN,NaN,0.091,...,Осн. режим: Облигации - Третий уровень,RU000A0JV052,2015-01-05 00:00:00,1780,1777,329.0,1,0.152215,-0.029015,1
552602,2015-01-06 00:00:00,2019-11-20 00:00:00,Московская Биржа,NaN,100.00,Market,NaN,NaN,NaN,0.091,...,Осн. режим: Облигации - Третий уровень,RU000A0JV052,2015-01-05 00:00:00,1780,1779,329.0,1,0.157476,-0.034276,1


In [47]:
df_short.groupby("ISIN").mean()[df_short.groupby("ISIN").mean().BA_spread.isnull()]

,BA_spread,Indicative_price,Turnover,Number_of_trades,Volume,Coupon,YTM_ind,Coupon_accum,Duration,Total_days_to_maturity,Days_to_maturity,Days_to_call,Callable,RF_interpolated,G_spread_interpolated
ISIN,,,,,,,,,,,,,,,
ROIIBKDBC023,NaN,100.540000,NaN,NaN,NaN,0.0340,0.031800,136.000000,558.545000,1054.0,584.000000,NaN,0.0,0.079748,-0.047948
ROIIBKDBC049,NaN,102.150000,NaN,NaN,NaN,0.0159,0.007800,49.120000,952.370000,983.0,983.000000,NaN,0.0,0.068338,-0.060538
ROSXSYT9K848,NaN,100.000000,NaN,NaN,NaN,0.0398,0.039733,103.186667,834.866667,974.0,879.666667,NaN,0.0,0.055694,-0.015960
ROXR6Y7LQ1H7,NaN,100.008889,NaN,NaN,NaN,0.0318,NaN,44.614444,NaN,1079.0,843.444444,NaN,0.0,0.068584,-0.026018
RU000A0JR0W3,NaN,100.000000,NaN,NaN,NaN,0.0900,0.091225,26.374824,121.497661,248.0,123.555556,NaN,0.0,0.112848,-0.021623
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
XS1324216768,NaN,80.922084,2.009499e+08,1.472222,9759.592593,0.0500,0.037548,2709.325292,216.709968,1104.0,278.779468,NaN,0.0,0.070244,0.331073
XS1503160571,NaN,43.811994,2.397059e+08,1.827957,10999.811111,0.0450,0.029860,2818.000000,136.749200,1091.0,637.400000,NaN,0.0,0.073697,0.360962
XS1946883342,NaN,105.797738,6.041793e+07,1.076923,NaN,0.0935,0.065890,183413.360476,759.455238,1279.0,849.857143,NaN,0.0,0.053836,0.012055


In [48]:
df_short[df_short.Turnover.isnull()]
df_short.groupby("ISIN").mean()[df_short.groupby("ISIN").mean().Turnover.isnull()]

,BA_spread,Indicative_price,Turnover,Number_of_trades,Volume,Coupon,YTM_ind,Coupon_accum,Duration,Total_days_to_maturity,Days_to_maturity,Days_to_call,Callable,RF_interpolated,G_spread_interpolated
ISIN,,,,,,,,,,,,,,,
HK0000138237,315.130933,95.256888,NaN,NaN,NaN,0.0360,0.171523,917.898802,191.181014,399.0,193.454172,NaN,0.0,0.104197,0.067326
HU0000358825,119.910465,100.880186,NaN,NaN,NaN,0.0200,0.015997,90.178129,753.369261,1092.0,768.288684,NaN,0.0,0.057716,-0.041719
HU0000359245,212.141343,98.955000,NaN,NaN,NaN,0.0125,0.017653,56.758451,853.424613,1090.0,865.390845,NaN,0.0,0.051675,-0.034022
ROIIBKDBC015,471.875000,100.810000,NaN,NaN,NaN,0.0410,0.037921,40.335417,1002.797917,1094.0,1059.833333,NaN,0.0,0.101720,-0.063799
ROIIBKDBC023,NaN,100.540000,NaN,NaN,NaN,0.0340,0.031800,136.000000,558.545000,1054.0,584.000000,NaN,0.0,0.079748,-0.047948
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
XS0993279958,606.704545,87.414592,NaN,NaN,NaN,0.0900,0.209459,4602.057613,463.243539,682.0,505.689796,NaN,0.0,0.115231,0.094228
XS0996297544,NaN,101.166667,NaN,NaN,NaN,0.1350,0.137133,9350.000000,220.710000,1598.0,238.333333,NaN,0.0,0.068571,0.068563
XS1059697323,NaN,100.000000,NaN,NaN,NaN,0.1100,0.112736,14263.888182,418.976364,830.0,452.454545,NaN,0.0,0.099822,0.012914


In [49]:
df_short["Month"] = df_short.Date_trading.apply(lambda x: datetime.strptime(re.sub(" 00:00:00", "", x), "%Y-%m-%d").month)
df_short["Year"] = df_short.Date_trading.apply(lambda x: datetime.strptime(re.sub(" 00:00:00", "", x), "%Y-%m-%d").year)

In [50]:
df_short.loc[df_short.cregnum == 354, "cregnum"] = "354"

In [51]:
df_short.cregnum = df_short.cregnum.apply(lambda x: int(re.sub(" ", "", x)))

In [52]:
df_short.cregnum.unique()

array([2306, 2879, 1810, 1975, 2590, 1326,  967, 3176,  436, 2312, 3255,
       2216,   30,  323, 2748, 3038, 3261, 1460, 3287, 1000,  354, 1942,
       3338, 2738,  918, 2495,   65, 3360, 3311, 2707, 3395,  204, 2618,
       2956, 2440, 2443, 3344,   77,  912, 1978, 3340, 3421, 2546, 2913,
       3437, 1521, 2766, 3251, 2110, 1776, 2412, 3292, 3354,  170, 2272,
       3073, 1573, 3349, 1354, 2790, 3123, 1073, 2289, 1792, 1481, 1961,
       1637, 3368,  963, 1555,  554, 2307, 2999, 3058, 2673, 3470, 2210,
        429, 2519, 2157, 2209, 3500,  316, 2225, 2998,    1], dtype=int64)

In [53]:
df_short_grouped = df_short.drop("cregnum", axis = 1).groupby(["ISIN","Exch_name", "Month", "Year"]).mean()
df_short_grouped = df_short_grouped.reset_index()
df_short_grouped_categorical = df_short[["Date_final", "Exch_name", "Trade_regime", "Date_first_trading", "ISIN", "Month", "Year", "cregnum"]].groupby(["ISIN","Exch_name", "Month", "Year"]).agg(lambda x: stats.mode((x))[0][0])
df_short_grouped_categorical = df_short_grouped_categorical.reset_index()
df_short_grouped = df_short_grouped.merge(df_short_grouped_categorical, on = ["ISIN", "Exch_name", "Year", "Month"])


In [54]:
df_short_grouped["NumberOfInstances"] = df_short.groupby(["ISIN","Exch_name", "Month", "Year"]).count().Date_trading.values

In [55]:
mean_turnover_ISIN = df_short_grouped.groupby(["ISIN"]).mean()["Turnover"]
mean_not_ISIN = df_short_grouped.groupby(["ISIN"]).mean()["Number_of_trades"]
mean_volume_ISIN = df_short_grouped.groupby(["ISIN"]).mean()["Volume"]
mean_BA_spread_ISIN = df_short_grouped.groupby(["ISIN"]).mean()["BA_spread"]
mean_duration_ISIN = df_short_grouped.groupby(["ISIN"]).mean()["Duration"]


In [56]:
df_short_grouped = df_short_grouped.merge(mean_turnover_ISIN, on = "ISIN", suffixes = ("", "_y"))
df_short_grouped = df_short_grouped.merge(mean_not_ISIN, on = "ISIN", suffixes = ("", "_y"))
df_short_grouped = df_short_grouped.merge(mean_volume_ISIN, on = "ISIN", suffixes = ("", "_y"))
df_short_grouped = df_short_grouped.merge(mean_BA_spread_ISIN, on = "ISIN", suffixes = ("", "_y"))
df_short_grouped = df_short_grouped.merge(mean_duration_ISIN, on = "ISIN", suffixes = ("", "_y"))

In [57]:
df_short_grouped.loc[df_short_grouped.Turnover.isnull(), "Turnover"] =\
df_short_grouped.loc[df_short_grouped.Turnover.isnull()].Turnover_y
df_short_grouped.loc[df_short_grouped.Turnover.isnull(), "Number_of_trades"] =\
df_short_grouped.loc[df_short_grouped.Number_of_trades.isnull()].Number_of_trades_y
df_short_grouped.loc[df_short_grouped.Volume.isnull(), "Volume"] =\
df_short_grouped.loc[df_short_grouped.Volume.isnull()].Volume_y
df_short_grouped.loc[df_short_grouped.BA_spread.isnull(), "BA_spread"] =\
df_short_grouped.loc[df_short_grouped.BA_spread.isnull()].BA_spread_y
df_short_grouped.loc[df_short_grouped.Duration.isnull(), "Duration"] =\
df_short_grouped.loc[df_short_grouped.Duration.isnull()].Duration_y

In [58]:
df_short_grouped.drop(["Number_of_trades", "Number_of_trades_y", "Turnover_y", "Volume_y", "BA_spread_y", "Duration_y"], axis = 1, inplace = True)

In [59]:
df_short_grouped.loc[df_short_grouped.Days_to_call.isnull(), "Days_to_call"] = 0

In [60]:
df_short_grouped.loc[(df_short_grouped.Turnover.isnull()), "Turnover"] = df_short_grouped.Turnover.mean()
df_short_grouped.loc[(df_short_grouped.Volume.isnull()), "Volume"] = df_short_grouped.Volume.mean()
df_short_grouped.loc[(df_short_grouped.BA_spread.isnull()), "BA_spread"] = df_short_grouped.BA_spread.mean()
df_short_grouped.loc[(df_short_grouped.Duration.isnull()), "Duration"] = 0


In [61]:
df_short_grouped.groupby("Exch_name").count()

,ISIN,Month,Year,BA_spread,Indicative_price,Turnover,Volume,Coupon,YTM_ind,Coupon_accum,...,Days_to_maturity,Days_to_call,Callable,RF_interpolated,G_spread_interpolated,Date_final,Trade_regime,Date_first_trading,cregnum,NumberOfInstances
Exch_name,,,,,,,,,,,,,,,,,,,,,
FINRA TRACE,367,367,367,367,367,367,367,362,337,367,...,367,367,367,367,367,367,367,367,367,367
Fixing HKMA,33,33,33,33,33,33,33,33,33,33,...,33,33,33,33,33,33,33,33,33,33
Gettex,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
"MiFID II Source 1 (APA, Post-trade reporting)",44,44,44,44,44,44,44,44,29,44,...,44,44,44,44,44,44,44,44,44,44
"MiFID II Source 2 (APA, Post-trade reporting)",29,29,29,29,29,29,29,29,19,29,...,29,29,29,29,29,29,29,29,29,29
Muscat Securities Market,1,1,1,1,1,1,1,1,0,0,...,1,1,1,1,1,1,1,1,1,1
Quotrix,10,10,10,10,10,10,10,10,10,10,...,10,10,10,10,10,10,10,10,10,10
TradeGate,21,21,21,21,21,21,21,21,21,21,...,21,21,21,21,21,21,21,21,21,21
Бакинская ФБ,13,13,13,13,13,13,13,13,0,0,...,13,13,13,13,13,13,13,13,13,13


In [62]:
df_short_grouped.Exch_name.unique()

array(['Берлинская биржа', 'Франкфуртская ФБ', 'Швейцарская ФБ',
       'Штутгартская ФБ', 'Будапештская ФБ', 'Бухарестская ФБ',
       'Московская Биржа', 'Московская Биржа. Внебиржевые сделки',
       'Московская Биржа. РПС', 'Московская биржа. РПС с ЦК',
       'Московская биржа. РЕПО с ЦК', 'Московская Биржа. РЕПО',
       'Московская биржа Т+', 'Фиксинг НФА', 'Бакинская ФБ',
       'Московская биржа. OTC-система',
       'MiFID II Source 2 (APA, Post-trade reporting)',
       'Санкт-Петербургская биржа. Внебиржевые сделки', 'СПВБ',
       'Братиславская ФБ', 'FINRA TRACE', 'Лондонская ФБ', 'Quotrix',
       'Дюссельдорфская ФБ', 'Мюнхенская ФБ', 'Fixing HKMA',
       'MiFID II Source 1 (APA, Post-trade reporting)', 'TradeGate',
       'Санкт-Петербургская биржа', 'Gettex', 'Muscat Securities Market',
       'Венская ФБ'], dtype=object)

In [63]:
df_short_grouped["MoscowSE"] = 0
df_short_grouped.loc[(df_short_grouped.Exch_name == 'Московская биржа Т+') | \
             (df_short_grouped.Exch_name == "Московская Биржа. Внебиржевые сделки") | \
             (df_short_grouped.Exch_name == "Московская Биржа") | \
             (df_short_grouped.Exch_name == "Московская биржа. РЕПО с ЦК") | \
             (df_short_grouped.Exch_name == "Московская Биржа. РПС") | \
             (df_short_grouped.Exch_name == "Московская Биржа. РЕПО")| \
             (df_short_grouped.Exch_name == "Московская биржа. OTC-система")| \
             (df_short_grouped.Exch_name == "Московская Биржа. РЕПО"), "MoscowSE"] = 1


In [64]:
df_short_grouped

,ISIN,Exch_name,Month,Year,BA_spread,Indicative_price,Turnover,Volume,Coupon,YTM_ind,...,Days_to_call,Callable,RF_interpolated,G_spread_interpolated,Date_final,Trade_regime,Date_first_trading,cregnum,NumberOfInstances,MoscowSE
0,CH0141533403,Берлинская биржа,1,2015,140.190476,94.044286,4.700925e+05,10000.000000,0.0500,0.129643,...,0.0,0.0,0.149253,-0.019611,2015-11-17 00:00:00,-,2015-01-01 00:00:00,1000,21,0
1,CH0141533403,Берлинская биржа,2,2015,59.411765,96.804750,4.700925e+05,23333.333333,0.0500,0.095280,...,0.0,0.0,0.129556,-0.034276,2015-11-17 00:00:00,-,2015-01-01 00:00:00,1000,20,0
2,CH0141533403,Берлинская биржа,3,2015,59.636364,97.705909,4.700925e+05,20000.000000,0.0500,0.085423,...,0.0,0.0,0.126801,-0.041378,2015-11-17 00:00:00,-,2015-01-01 00:00:00,1000,22,0
3,CH0141533403,Берлинская биржа,4,2015,62.625000,99.249750,4.700925e+05,1.000000,0.0500,0.061780,...,0.0,0.0,0.116360,-0.054580,2015-11-17 00:00:00,-,2015-01-01 00:00:00,1000,20,0
4,CH0141533403,Берлинская биржа,5,2015,57.352941,99.128500,4.700925e+05,1.000000,0.0500,0.067515,...,0.0,0.0,0.103328,-0.035813,2015-11-17 00:00:00,-,2015-01-01 00:00:00,1000,20,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29615,XS2107554896,"MiFID II Source 1 (APA, Post-trade reporting)",9,2020,580.448688,101.150000,1.098900e+08,310382.806157,0.0675,0.065400,...,0.0,0.0,0.054412,0.010988,2025-01-30 00:00:00,-,2020-01-24 00:00:00,1326,1,0
29616,XS2107554896,"MiFID II Source 1 (APA, Post-trade reporting)",11,2020,580.448688,100.755000,1.098900e+08,310382.806157,0.0675,0.066425,...,0.0,0.0,0.051730,0.014695,2025-01-30 00:00:00,-,2020-01-24 00:00:00,1326,4,0
29617,XS2107554896,"MiFID II Source 1 (APA, Post-trade reporting)",12,2020,580.448688,101.136667,1.098900e+08,310382.806157,0.0675,0.065333,...,0.0,0.0,0.051324,0.014009,2025-01-30 00:00:00,-,2020-01-24 00:00:00,1326,3,0
29618,XS2107554896,"MiFID II Source 2 (APA, Post-trade reporting)",7,2020,580.448688,101.486000,1.098900e+08,310382.806157,0.0675,0.064680,...,0.0,0.0,0.050931,0.013749,2025-01-30 00:00:00,-,2020-01-24 00:00:00,1326,5,0


In [69]:
df_short_grouped.drop("Exch_name", axis = 1, inplace = True)

In [66]:
df_short_grouped["MonthIssued"] = df_short_grouped.Date_first_trading.apply(lambda x: datetime.strptime(re.sub(" 00:00:00", "", x), "%Y-%m-%d").month)
df_short_grouped["YearIssued"] = df_short_grouped.Date_first_trading.apply(lambda x: datetime.strptime(re.sub(" 00:00:00", "", x), "%Y-%m-%d").year)

In [67]:
df_short_grouped.drop(["Date_first_trading", "Date_final", "Trade_regime"], axis = 1, inplace = True)

In [73]:
df_short_grouped.to_csv("C:\\Users\\Ivand\\Repos\\banks\\data\\BondsPrepared.csv")